In [ ]:
import h5py
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from lv17analysis import lv17data
from os import path, makedirs

## Helper functions for easy readout and plotting

In [ ]:

def golden_figsize(fwidth=8):
    golden_ratio = 1.618034
    fheight = fwidth/golden_ratio
    return (fwidth, fheight)


def read_data(run):
    data_dir = '/reg/data/ana01/tmo/tmolv1720/hdf5/smalldata/'
    data_fname = 'r{:03d}_epix.h5'.format(run)

    data_keys = ['epix_lit', 'epix_sum', 'gmd', 'photon_energy',
                'source_delay', 'source_on', 'source_temp',
                'timestamp', 'tof_abs_sum', 'tof_sum', 'xgmd']
    data = {}
    with h5py.File(data_dir + data_fname, 'r') as f:
        for key in data_keys:
            data[key] = np.copy(f[key])
    data['run'] = run
    return data


def data_filter(data, source='on',
                gmd_min=0.0, gmd_max=np.inf,
                xgmd_min=0.0, xgmd_max=np.inf):
    source_on = data['source_on'] == True
    source_filt = np.array(source_on if source=='on' else ~source_on)
    gmd_filt = np.array(data['gmd'] >= 0) & \
        np.array(data['gmd'] <= np.inf)
    xgmd_filt = np.array(data['xgmd'] >= 0) & \
        np.array(data['xgmd'] <= np.inf)
    return source_filt & gmd_filt & xgmd_filt


def plot_data(data, dkey='epix_lit', bins=100, norm=None,
              figsize=(8,5), ptype='plot', yscale='linear'):
    dfilt = data_filter(data)

    if norm is None:
        plotdata = data[dkey]
    else:
        plotdata = data[dkey] / data[norm]

    fig = plt.figure(figsize=figsize)
    if ptype == 'plot':
        plt.plot(plotdata[dfilt], label='source on')
        plt.plot(plotdata[~dfilt], label='source off')
    elif ptype == 'hist':
        plt.hist(plotdata[dfilt], bins=100, label='source on', alpha=0.7)
        plt.hist(plotdata[~dfilt], bins=100, label='source off', alpha=0.7)
    plt.yscale(yscale)
    plt.legend(title='run {:03d}, {}'.format(data['run'], dkey))
    return fig


def save_plot(fig, save_name,
               save_dir='/reg/d/psdm/tmo/tmolv1720/results/run_hist/'):
    makedirs(save_dir, exist_ok=True)
    fname = path.join(save_dir, save_name)
    plt.savefig(fname)


def plot_and_save(data, dkey='epix_lit', bins=100, norm=None,
                  figsize=(8,5), ptype='plot', closefig=False,
                  yscale='linear'):
    fig = plot_data(data=data, dkey=dkey, bins=bins, norm=norm,
                    figsize=figsize, ptype=ptype, yscale=yscale)
    save_name = "r{:3d}_{}_{}.png".format(data['run'], dkey, ptype)
    save_plot(fig, save_name)
    if closefig:
        plt.close(fig=fig)

## Create plots and histograms for all epix runs

In [ ]:
run_list = lv17data.epix_runs

for run in run_list:
    data = read_data(run=run)
    for dkey in ['epix_lit', 'epix_sum', 'tof_abs_sum', 'gmd', 'xgmd']:
        for ptype in ['plot', 'hist']:
            if ptype == 'hist' and dkey not in ['gmd', 'xgmd']:
                yscale = 'log'
            else:
                yscale = 'linear'
            plot_and_save(data, dkey=dkey, ptype=ptype, closefig=True,
                          yscale=yscale)